In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
import joblib
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest,f_regression
import pickle

import json
from pathlib import Path
from xgboost import XGBRegressor

In [3]:
#!pip install shap

In [5]:
dataset = pd.read_csv('electricity_bill_dataset.csv')

In [7]:
dataset

,Fan,Refrigerator,AirConditioner,Television,Monitor,MotorPump,Month,City,Company,MonthlyHours,TariffRate,ElectricityBill
0,16,23.0,2.0,6.0,1.0,0,10,Hyderabad,Tata Power Company Ltd.,384,8.4,3225.6
1,19,22.0,2.0,3.0,1.0,0,5,Vadodara,NHPC,488,7.8,3806.4
2,7,20.0,2.0,6.0,7.0,0,7,Shimla,Jyoti Structure,416,7.7,3203.2
3,7,22.0,3.0,21.0,1.0,0,6,Mumbai,Power Grid Corp,475,9.2,4370.0
4,11,23.0,2.0,11.0,1.0,0,2,Mumbai,Ratnagiri Gas and Power Pvt. Ltd. (RGPPL),457,9.2,4204.4
...,...,...,...,...,...,...,...,...,...,...,...,...
45340,18,22.0,3.0,22.0,1.0,0,9,Ahmedabad,Maha Transco – Maharashtra State Electricity T...,764,7.9,6035.6
45341,23,23.0,2.0,6.0,12.0,0,2,New Delhi,Orient Green,572,8.5,4862.0
45342,22,22.0,2.0,20.0,1.0,0,1,New Delhi,GE T&D India Limited,609,8.5,5176.5
45343,8,21.0,2.0,22.0,7.0,0,12,Ratnagiri,TransRail Lighting,748,7.4,5535.2


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fan              45345 non-null  int64  
 1   Refrigerator     45345 non-null  float64
 2   AirConditioner   45345 non-null  float64
 3   Television       45345 non-null  float64
 4   Monitor          45345 non-null  float64
 5   MotorPump        45345 non-null  int64  
 6   Month            45345 non-null  int64  
 7   City             45345 non-null  object 
 8   Company          45345 non-null  object 
 9   MonthlyHours     45345 non-null  int64  
 10  TariffRate       45345 non-null  float64
 11  ElectricityBill  45345 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 4.2+ MB


In [11]:
dataset.isnull().sum()

Fan                0
Refrigerator       0
AirConditioner     0
Television         0
Monitor            0
MotorPump          0
Month              0
City               0
Company            0
MonthlyHours       0
TariffRate         0
ElectricityBill    0
dtype: int64

In [13]:
df=dataset

In [15]:
df = df.drop(columns=['Company'])
df = df.drop(columns=['City'])

In [17]:
df['AirConditioner'].value_counts()

AirConditioner
2.0    11474
1.0    11328
3.0    11307
0.0    11236
Name: count, dtype: int64

In [19]:
df['MotorPump'].value_counts()

MotorPump
0    45345
Name: count, dtype: int64

In [21]:
df

,Fan,Refrigerator,AirConditioner,Television,Monitor,MotorPump,Month,MonthlyHours,TariffRate,ElectricityBill
0,16,23.0,2.0,6.0,1.0,0,10,384,8.4,3225.6
1,19,22.0,2.0,3.0,1.0,0,5,488,7.8,3806.4
2,7,20.0,2.0,6.0,7.0,0,7,416,7.7,3203.2
3,7,22.0,3.0,21.0,1.0,0,6,475,9.2,4370.0
4,11,23.0,2.0,11.0,1.0,0,2,457,9.2,4204.4
...,...,...,...,...,...,...,...,...,...,...
45340,18,22.0,3.0,22.0,1.0,0,9,764,7.9,6035.6
45341,23,23.0,2.0,6.0,12.0,0,2,572,8.5,4862.0
45342,22,22.0,2.0,20.0,1.0,0,1,609,8.5,5176.5
45343,8,21.0,2.0,22.0,7.0,0,12,748,7.4,5535.2


In [23]:
independent = df.drop("ElectricityBill", axis=1)
dependent = df["ElectricityBill"]

In [25]:
independent

,Fan,Refrigerator,AirConditioner,Television,Monitor,MotorPump,Month,MonthlyHours,TariffRate
0,16,23.0,2.0,6.0,1.0,0,10,384,8.4
1,19,22.0,2.0,3.0,1.0,0,5,488,7.8
2,7,20.0,2.0,6.0,7.0,0,7,416,7.7
3,7,22.0,3.0,21.0,1.0,0,6,475,9.2
4,11,23.0,2.0,11.0,1.0,0,2,457,9.2
...,...,...,...,...,...,...,...,...,...
45340,18,22.0,3.0,22.0,1.0,0,9,764,7.9
45341,23,23.0,2.0,6.0,12.0,0,2,572,8.5
45342,22,22.0,2.0,20.0,1.0,0,1,609,8.5
45343,8,21.0,2.0,22.0,7.0,0,12,748,7.4


In [27]:
dependent

0        3225.6
1        3806.4
2        3203.2
3        4370.0
4        4204.4
          ...  
45340    6035.6
45341    4862.0
45342    5176.5
45343    5535.2
45344    3544.1
Name: ElectricityBill, Length: 45345, dtype: float64

In [44]:
x_train,x_test,y_train,y_test = train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [46]:
x_train

,Fan,Refrigerator,AirConditioner,Television,Monitor,MotorPump,Month,MonthlyHours,TariffRate
29548,8,20.0,1.0,9.0,1.0,0,7,407,8.9
6481,19,22.0,3.0,20.0,1.0,0,6,627,8.5
42807,13,23.0,1.0,8.0,1.0,0,2,493,7.4
8613,5,22.0,1.0,18.0,1.0,0,7,504,8.8
6317,12,23.0,1.0,9.0,1.0,0,1,441,9.2
...,...,...,...,...,...,...,...,...,...
30403,14,23.0,1.0,7.0,1.0,0,5,476,7.8
21243,22,22.0,1.0,6.0,12.0,0,1,716,9.3
42613,18,23.0,0.0,14.0,1.0,0,8,535,8.5
43567,9,23.0,1.0,18.0,1.0,0,3,573,8.9


In [48]:
x_test

,Fan,Refrigerator,AirConditioner,Television,Monitor,MotorPump,Month,MonthlyHours,TariffRate
17052,17,23.0,1.0,13.0,1.0,0,6,413,8.7
7961,9,19.0,0.0,12.0,1.0,0,7,377,9.1
39705,7,20.0,2.0,13.0,1.0,0,12,323,9.3
13523,7,20.0,0.0,16.0,1.0,0,9,397,8.4
11833,5,22.0,0.0,15.0,1.0,0,12,352,9.1
...,...,...,...,...,...,...,...,...,...
28543,21,22.0,2.0,17.0,12.0,0,2,743,9.3
173,7,17.0,1.0,18.0,7.0,0,11,536,9.3
23832,23,22.0,0.0,22.0,1.0,0,8,738,8.1
22414,22,23.0,3.0,9.0,12.0,0,2,529,7.9


In [50]:
Regressor = GradientBoostingRegressor(random_state=0, n_estimators=200, learning_rate=0.5, max_depth=3)
model = Regressor.fit(x_train,y_train)

In [52]:
model

GradientBoostingRegressor(learning_rate=0.5, n_estimators=200, random_state=0)

In [54]:
importances = model.feature_importances_
feature_names = independent.columns

In [56]:
# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

In [58]:
print("\nFeature Importances:")
print(importance_df)



Feature Importances:
          Feature    Importance
7    MonthlyHours  9.218408e-01
8      TariffRate  7.815176e-02
0             Fan  3.523424e-06
3      Television  2.940941e-06
6           Month  5.232467e-07
1    Refrigerator  3.202721e-07
4         Monitor  9.001833e-08
2  AirConditioner  1.051857e-08
5       MotorPump  0.000000e+00


In [60]:
y_pred = model.predict(x_test)     

In [62]:

mse = mean_squared_error(y_test, y_pred)
r_score = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R2): {r_score:f}")

Mean Squared Error (MSE): 298.90
R-squared (R2): 0.999738


In [64]:
y_pred

array([3607.23288384, 3404.73222408, 3009.55339358, ..., 5974.46831257,
       4192.48006032, 4516.34729722])

In [66]:
r_score

0.9997377238264767

In [68]:
mse

298.8997994741312

In [73]:
joblib.dump(model, 'model.pkl')
st.sidebar.success(f"Model trained (MAE ≈ ₹{mse:,.0f}) and saved to model.pkl")

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())